In [92]:
import os
import numpy as np
from PIL import Image
import random
import math
import pandas as pd
import cv2
import pandas as pd
from tqdm import tqdm

In [93]:
# 1. Carica l’immagine e normalizza in [0,1]
img = Image.open('cat.jpg').convert('RGB')
img_arr = np.asarray(img, dtype=np.float32) / 255.0


b = 0.17
a = -b
noisy_arr = add_salt_pepper_noise(img_arr)
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
# 4a. Salva su disco
#noisy_img.save('noisy_image.jpg')

# 4b. Oppure mostra a video (apre il viewer predefinito)
noisy_img.show()
#Andiamo

In [94]:
def add_salt_pepper_noise(image: np.ndarray, amount: float = 0.05, salt_vs_pepper: float = 0.5) -> np.ndarray:
    """
    :param image: Input image as float32 numpy array in [0,1], shape (H, W, 3).
    :param amount: Fraction of total pixels to corrupt.
    :param salt_vs_pepper: Fraction of corrupted pixels that are salt (vs pepper).
    :return: Noisy image as numpy array.
    """
    noisy = np.copy(image)
    ROWS, COLS, _ = image.shape
    num_pixels = ROWS * COLS
    num_salt = int(np.ceil(amount * num_pixels * salt_vs_pepper))
    num_pepper = int(np.ceil(amount * num_pixels * (1 - salt_vs_pepper)))

    # SALT
    coords_y = np.random.randint(0, ROWS, num_salt)
    coords_x = np.random.randint(0, COLS, num_salt)
    noisy[coords_y, coords_x, :] = 1.0

    # PEPPER
    coords_y = np.random.randint(0, ROWS, num_pepper)
    coords_x = np.random.randint(0, COLS, num_pepper)
    noisy[coords_y, coords_x, :] = 0.0

    return noisy

rng = np.random.default_rng()


def add_gaussian_noise(image: np.ndarray, mean=0.0, var=0.01):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param mean: Mean of Gaussian noise.
    :param var: Variance of Gaussian noise.
    :return: Noisy image as numpy array.
    """
    sigma = np.float32(var**0.5)
    noisy = image.copy()  # float32
    rng.standard_normal(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= sigma
    noisy += image + np.float32(mean)
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy


def add_uniform_noise(image: np.ndarray, low=-0.1732, high=0.1732):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param low: Lower bound of uniform noise.
    :param high: Upper bound of uniform noise.
    :return: Noisy image as numpy array.
    """
    noisy = image.copy()  # float32
    rng.random(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= np.float32(high - low)
    noisy += np.float32(low) + image
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy



def add_erlang_noise(image: np.ndarray, k=3, lam=17.32):
    """
    Add Erlang (Gamma) noise to an image, using shape k and rate λ.
    The Erlang PDF is: f(x) = (λ^k * x^(k-1) / (k-1)!) * e^{-λ x},  x >= 0

    :param image:     Input image as float32 numpy array in [0,1], shape (H, W, C).
    :param k:         Shape parameter (integer ≥ 1).
    :param lam:       Rate parameter λ (> 0).
    :return:          Noisy image as numpy array, clipped to [0,1].
    """
    theta = np.float32(1.0/lam)
    noisy = image.copy() # float32
    noise64 = rng.gamma(shape=k, scale=theta, size=image.shape) # genero rumore in float64 (per forza)
    noise = noise64.astype(np.float32) # cast to float32 (necessario)
    noise -= (np.float32(k) * theta)
    noisy += noise
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy



def add_periodic_noise(image: np.ndarray, A: float = 0.05, fx: float = 0.05, fy: float = 0.07, phi: float = 0.0) -> np.ndarray:
    """
    Aggiunge un rumore periodico (sinusoidale) su ciascun canale di un'immagine RGB.

    :param image: Input image come array numpy float32 in [0,1], shape (H, W, 3).
    :param A:     Ampiezza del rumore.
    :param fx:    Frequenza orizzontale (cicli/pixel).
    :param fy:    Frequenza verticale (cicli/pixel).
    :param phi:   Fase iniziale del seno (in radianti).
    :return:      Immagine con rumore aggiunto, sempre clip-pata in [0,1].
    """
    # Copia dell'immagine originale
    noisy = image.copy()

    M, N, C = image.shape  # M = righe (height), N = colonne (width), C = canali (RGB)

    # Costruiamo la griglia X, Y in modo analogo a meshgrid(1:N, 1:M) di MATLAB
    # Nota: in MATLAB gli indici partono da 1, quindi usiamo arange(1, N+1) e (1, M+1)
    x_coords = np.arange(1, N + 1, dtype=np.float32)  # 1, 2, ..., N
    y_coords = np.arange(1, M + 1, dtype=np.float32)  # 1, 2, ..., M
    X, Y = np.meshgrid(x_coords, y_coords)            # X e Y hanno shape (M, N)

    # Rumore 2D: A * sin(2*pi*(fx * X + fy * Y) + phi)
    noise2D = A * np.sin(2 * np.pi * (fx * X + fy * Y) + phi)  # shape (M, N)

    # Estendiamo il rumore su tutti e 3 i canali: diventa (M, N, 3)
    noise3Ch = np.repeat(noise2D[:, :, np.newaxis], 3, axis=2)

    # Aggiungiamo il rumore e riportiamo nell’intervallo [0,1]
    noisy += noise3Ch
    np.clip(noisy, 0.0, 1.0, out=noisy)

    return noisy



def add_rayleigh_noise(image: np.ndarray, sigma: float = 0.1525) -> np.ndarray:
    """
    Aggiunge rumore Rayleigh a un'immagine, riportando il rumore a media zero
    come nel codice MATLAB fornito.

    :param image: Input image come array numpy float32 in [0,1], shape (H, W, C).
    :param sigma: Parametro di scala del rumore Rayleigh (per varianza ≈ 0.01, usato 0.1525).
    :return:      Immagine con rumore Rayleigh zero‐mean, clip‐pata in [0,1].
    """
    # Copia dell'immagine originale
    noisy = image.copy()

    # Genera rumore Rayleigh (float64 di default), poi cast a float32
    rayleigh = rng.rayleigh(scale=sigma, size=image.shape).astype(np.float32)

    # Calcola la media del Rayleigh (σ * sqrt(pi/2)) e sottraila
    mean_shift = np.float32(sigma * np.sqrt(np.pi / 2))
    noise_zero_mean = rayleigh - mean_shift

    # Aggiungi il rumore e riportalo nell'intervallo [0,1]
    noisy += noise_zero_mean
    np.clip(noisy, 0.0, 1.0, out=noisy)

    return noisy


def add_speckle_noise(image: np.ndarray, var: float = 0.01) -> np.ndarray:
    """
    Aggiunge rumore speckle (moltiplicativo) a un'immagine come fa imnoise(..., 'speckle', var) in MATLAB.

    :param image: Input image come array numpy float32 o float64 in [0,1], shape (H, W, C) o (H, W).
    :param var:   Varianza del rumore speckle (lo stesso parametro passato a imnoise in MATLAB).
    :return:      Immagine con rumore speckle, clip-pata in [0,1].
    """
    # Assicuriamoci che l'immagine sia in float32 per coerenza
    img = image.astype(np.float32)

    # Genera rumore gaussiano standard (media 0, varianza 1) nella stessa shape dell'immagine
    noise = rng.standard_normal(size=img.shape, dtype=np.float32)

    # Rumore speckle: J = I + sqrt(var) * I .* noise
    noisy = img + np.sqrt(var).astype(np.float32) * img * noise

    # Clip dei valori nell'intervallo [0,1]
    np.clip(noisy, 0.0, 1.0, out=noisy)

    return noisy



def add_vertical_striping_noise(image: np.ndarray, amplitude: float = 0.15, frequency: float = 25.0) -> np.ndarray:
    """
    Aggiunge rumore a bande verticali a un'immagine RGB normalizzata in [0,1].
    Equivalente al codice MATLAB:
        [yGrid, ~] = meshgrid(1:cols, 1:rows);
        stripe_pattern = amplitude * sin(2*pi*frequency*yGrid/rows);
        I_noisy(:,:,c) = mat2gray(I(:,:,c) + stripe_pattern);
    
    :param image:     array numpy float32 o float64 in [0,1], shape (H, W, 3).
    :param amplitude: ampiezza del pattern sinusoidale (default 0.15).
    :param frequency: frequenza (numero di cicli in base alla dimensione orizzontale, default 25).
    :return:          immagine con rumore a bande verticali, tipo float32 in [0,1].
    """
    # Recupera dimensioni
    H, W, C = image.shape

    # Crea yGrid come in MATLAB: mat2grid(1:cols, 1:rows) restituisce X con shape (H,W),
    # dove X[i,j] = j+1. Usiamo 1-based indices per restare fedeli al MATLAB.
    y = np.arange(1, W + 1, dtype=np.float32)        # 1, 2, ..., W
    yGrid = np.tile(y[np.newaxis, :], (H, 1))         # shape (H, W)

    # Costruisci il pattern sinusoidale
    # Nota: in MATLAB viene diviso per rows=H
    stripe_pattern = amplitude * np.sin(2 * np.pi * frequency * yGrid / H)  # (H, W)

    # Prealloca l'output
    noisy = np.zeros_like(image, dtype=np.float32)

    # Per ogni canale, somma pattern e poi applica mat2gray (scalatura in [0,1])
    for c in range(C):
        channel = image[:, :, c].astype(np.float32) + stripe_pattern

        # mat2gray: (x - min) / (max - min), canale per canale
        min_val = channel.min()
        max_val = channel.max()
        if max_val > min_val:
            channel_norm = (channel - min_val) / (max_val - min_val)
        else:
            # Se il canale è costante dopo l'aggiunta (causa valori identici), lo portiamo a zero
            channel_norm = np.zeros_like(channel)

        noisy[:, :, c] = channel_norm

    return noisy


def add_horizontal_striping_noise(image: np.ndarray, amplitude: float = 0.15, frequency: float = 20.0) -> np.ndarray:
    """
    Aggiunge rumore a bande orizzontali a un'immagine RGB normalizzata in [0,1].
    Equivalente al codice MATLAB:
        [~, yGrid] = meshgrid(1:cols, 1:rows);
        stripe_pattern = amplitude * sin(2*pi*frequency*yGrid/rows);
        I_noisy(:,:,c) = mat2gray(I(:,:,c) + stripe_pattern);
    
    :param image:     array numpy float32 o float64 in [0,1], shape (H, W, 3).
    :param amplitude: ampiezza del pattern sinusoidale (default 0.15).
    :param frequency: frequenza (numero di cicli in base alla dimensione verticale, default 20).
    :return:          immagine con rumore a bande orizzontali, tipo float32 in [0,1].
    """
    H, W, C = image.shape

    # Crea yGrid come secondo output di meshgrid(1:cols, 1:rows)
    # In MATLAB: [~, Y] = meshgrid(1:cols, 1:rows); Y[i,j] = i+1 (1-based row index).
    y = np.arange(1, H + 1, dtype=np.float32)        # 1, 2, ..., H
    yGrid = np.tile(y[:, np.newaxis], (1, W))        # shape (H, W)

    # Costruisci il pattern sinusoidale
    stripe_pattern = amplitude * np.sin(2 * np.pi * frequency * yGrid / H)

    noisy = np.zeros_like(image, dtype=np.float32)

    # Per ogni canale, somma pattern e poi applica mat2gray
    for c in range(C):
        channel = image[:, :, c].astype(np.float32) + stripe_pattern

        # mat2gray: normalizza in [0,1]
        min_val = channel.min()
        max_val = channel.max()
        if max_val > min_val:
            channel_norm = (channel - min_val) / (max_val - min_val)
        else:
            channel_norm = np.zeros_like(channel)

        noisy[:, :, c] = channel_norm

    return noisy

In [95]:
# 1. Carica e normalizza
img = Image.open('peppers.png').convert('RGB')
arr = np.asarray(img, dtype=np.float32) / 255.0

noisy_arr = add_vertical_striping_noise(arr,0.05,25)
# 12,25
# 3. Riconverti e salva
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
noisy_img.show()
#noisy_img.save('noisy_stripes.jpg')

In [96]:
# fx da 0.005 A 0.1 0 -0.005 a -0.1, A da 0.05 a 0.09
# striping: amplitude tra 0.05 e 0.15, frequency tra 25 e 75
# rayleigh: sigma da 0.1 a 0.2 
# speckle: var da 0.009 a 0.02

Bisogna calcolare la varianza per ogni rumore, e settare i parametri affinchè la varianza media sia 0.01.

In [97]:
import os
import random
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

def generate_dataset(input_dir: str, output_dir: str, batch_size: int = 32) -> None:
    """
    Genera un dataset rumoroso a partire da una cartella di immagini di input, salvando
    nove classi diverse in sottocartelle separate:
      - original
      - salt_pepper
      - gaussian
      - uniform
      - erlang_rayleigh   (contiene sia rumore Erlang che Rayleigh)
      - periodic
      - speckle
      - striping_vertical
      - striping_horizontal

    Per ciascuna immagine di input, viene salvata:
      - l’immagine originale
      - la versione con rumore salt & pepper (amount ∈ [0.01, 0.10])
      - la versione con rumore Gaussiano (varianza ∈ [0.005, 0.02])
      - la versione con rumore Uniforme (ampiezza ∈ [0.10, 0.20])
      - la versione con rumore Erlang *o* Rayleigh (50% di probabilità ciascuno; λ ∈ [12, 25] per Erlang, σ ∈ [0.10, 0.20] per Rayleigh)
      - la versione con rumore Periodic (A ∈ [0.05,0.09], fx,fy ∈ [0.005,0.1] con segno ± casuale, φ=0)
      - la versione con rumore Speckle (varianza ∈ [0.007,0.02])
      - la versione con rumore Striping Verticale (ampiezza ∈ [0.05,0.15], frequenza ∈ [25,75])
      - la versione con rumore Striping Orizzontale (ampiezza ∈ [0.05,0.15], frequenza ∈ [25,75])

    “Erlang” e “Rayleigh” vengono salvati nella stessa cartella “erlang_rayleigh” e etichettati con la stessa label.
    Alla fine viene prodotto un CSV “labels.csv” contenente filepath e label.
    """

    # Definizione degli intervalli di parametro
    sp_range     = (0.01, 0.10)
    gauss_range  = (0.005, 0.02)
    uni_range    = (0.10, 0.20)
    erlang_range = (12.0, 25.0)
    rayleigh_range = (0.10, 0.20)
    periodic_A_range = (0.05, 0.09)
    periodic_f_range = (0.005, 0.10)
    strip_amp_range = (0.05, 0.15)
    strip_freq_range = (25.0, 75.0)
    speckle_range = (0.009, 0.02)
    k_erlang = 3

    # Nomi delle sottocartelle (classi)
    variants = [
        'original',
        'salt_pepper',
        'gaussian',
        'uniform',
        'erlang_rayleigh',
        'periodic',
        'speckle',
        'striping_vertical',
        'striping_horizontal'
    ]
    # Mappatura da nome cartella a label
    label_map = {
        'original':           'Original',
        'salt_pepper':        'Salt & Pepper',
        'gaussian':           'Gaussian',
        'uniform':            'Uniform',
        'erlang_rayleigh':    'Erlang/Rayleigh',
        'periodic':           'Periodic',
        'speckle':            'Speckle',
        'striping_vertical':  'Striping Vertical',
        'striping_horizontal':'Striping Horizontal'
    }

    # Crea le sottocartelle di output
    for v in variants:
        os.makedirs(os.path.join(output_dir, v), exist_ok=True)

    # Elenca tutti i file immagine nella cartella di input
    files = [
        f for f in os.listdir(input_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
    ]

    records = []
    pbar = tqdm(total=len(files), desc="Processing images")

    # Elaborazione a batch
    for i in range(0, len(files), batch_size):
        batch = files[i:i+batch_size]
        imgs = []
        # 1) Carica il batch di immagini col loro path
        for fn in batch:
            src = os.path.join(input_dir, fn)
            bgr = cv2.imread(src, cv2.IMREAD_COLOR)
            if bgr is None:
                # Salta file non leggibili
                continue
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            imgs.append((fn, rgb.astype(np.float32) / 255.0))

        # 2) Per ciascuna immagine del batch, applica i diversi rumori e salva
        for fn, img in imgs:
            # — Originale —
            out_orig = os.path.join(output_dir, 'original', fn)
            cv2.imwrite(
                out_orig,
                cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_orig, 'label': label_map['original']})

            # — Salt & Pepper —
            amt = random.uniform(*sp_range)
            sp = add_salt_pepper_noise(img, amount=amt)
            out_sp = os.path.join(output_dir, 'salt_pepper', fn)
            cv2.imwrite(
                out_sp,
                cv2.cvtColor((sp * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_sp, 'label': label_map['salt_pepper']})

            # — Gaussian —
            var_g = random.uniform(*gauss_range)
            g = add_gaussian_noise(img, var=var_g)
            out_g = os.path.join(output_dir, 'gaussian', fn)
            cv2.imwrite(
                out_g,
                cv2.cvtColor((g * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_g, 'label': label_map['gaussian']})

            # — Uniform —
            r = random.uniform(*uni_range)
            u = add_uniform_noise(img, low=-r, high=+r)
            out_u = os.path.join(output_dir, 'uniform', fn)
            cv2.imwrite(
                out_u,
                cv2.cvtColor((u * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_u, 'label': label_map['uniform']})

            # — Erlang o Rayleigh (50% probabilità) —
            if random.random() < 0.5:
                # Erlang
                lam = random.uniform(*erlang_range)
                e = add_erlang_noise(img, k=k_erlang, lam=lam)
            else:
                # Rayleigh
                sigma = random.uniform(*rayleigh_range)
                e = add_rayleigh_noise(img, sigma=sigma)

            out_e = os.path.join(output_dir, 'erlang_rayleigh', fn)
            cv2.imwrite(
                out_e,
                cv2.cvtColor((e * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_e, 'label': label_map['erlang_rayleigh']})

            # — Periodic —
            A_p = random.uniform(*periodic_A_range)
            fx_val = random.uniform(*periodic_f_range)
            # assegna segno ± con uguale probabilità
            if random.random() < 0.5:
                fx_val = -fx_val
            fy_val = random.uniform(*periodic_f_range)
            if random.random() < 0.5:
                fy_val = -fy_val
            p = add_periodic_noise(img, A=A_p, fx=fx_val, fy=fy_val, phi=0.0)
            out_p = os.path.join(output_dir, 'periodic', fn)
            cv2.imwrite(
                out_p,
                cv2.cvtColor((p * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_p, 'label': label_map['periodic']})

            # — Speckle —
            var_s = random.uniform(*speckle_range)
            s = add_speckle_noise(img, var=var_s)
            out_s = os.path.join(output_dir, 'speckle', fn)
            cv2.imwrite(
                out_s,
                cv2.cvtColor((s * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_s, 'label': label_map['speckle']})

            # — Striping Verticale —
            amp_sv = random.uniform(*strip_amp_range)
            freq_sv = random.uniform(*strip_freq_range)
            sv = add_vertical_striping_noise(img, amplitude=amp_sv, frequency=freq_sv)
            out_sv = os.path.join(output_dir, 'striping_vertical', fn)
            cv2.imwrite(
                out_sv,
                cv2.cvtColor((sv * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_sv, 'label': label_map['striping_vertical']})

            # — Striping Orizzontale —
            amp_sh = random.uniform(*strip_amp_range)
            freq_sh = random.uniform(*strip_freq_range)
            sh = add_horizontal_striping_noise(img, amplitude=amp_sh, frequency=freq_sh)
            out_sh = os.path.join(output_dir, 'striping_horizontal', fn)
            cv2.imwrite(
                out_sh,
                cv2.cvtColor((sh * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            )
            records.append({'filepath': out_sh, 'label': label_map['striping_horizontal']})

            # Aggiorna barra di progresso
            pbar.update(1)

    pbar.close()

    # Salva CSV delle etichette
    pd.DataFrame(records).to_csv(
        os.path.join(output_dir, 'labels.csv'),
        index=False
    )
    print(f"Dataset e labels salvati in '{output_dir}'")

In [98]:
# ESEGUO
input_directory = "CocoBase_10k/."
output_directory = "DATASET_10000x9"
generate_dataset(input_directory, output_directory)

Processing images: 100%|██████████| 10000/10000 [21:14<00:00,  7.85it/s]  

Dataset e labels salvati in 'DATASET_10000x9'


In [4]:
import time

# Benchmark
img = Image.open('cat.jpg').convert('RGB')
image = np.asarray(img, dtype=np.float32) / 255.0
functions = {
    'Salt & Pepper': lambda img: add_salt_pepper_noise(img),
    'Gaussian': lambda img: add_gaussian_noise(img),
    'Uniform': lambda img: add_uniform_noise(img),
    'Erlang (Gamma)': lambda img: add_erlang_noise(img)
}

N = 60
results = {}
for name, fn in functions.items():
    start = time.time()
    for _ in range(N):
        _ = fn(image)
    end = time.time()
    results[name] = end - start

for name, total in results.items():
    print(f"{name}: {total:.4f}s total, {total/N:.6f}s per run")


Salt & Pepper: 0.2359s total, 0.003932s per run
Gaussian: 1.3666s total, 0.022776s per run
Uniform: 0.6268s total, 0.010446s per run
Erlang (Gamma): 2.9143s total, 0.048572s per run
